# Telco Customer Churn

## Motivation:

Customer retention is important to any growing company because it measures how successful they are at satisfying existing customers.

Moreover, it is also well known that:
*  Retain customers is more cost-effective than acquiring new ones,
* Retained customers tend to spend more and buy more often,
* Retained customers are more willing to refer your products to friends and family.


## Objective:

The aim of this study is to analyze relevant customer data in order to develop focused customer retention programs.

Concretely, we'd like to answer the following questions:

1. Is the percentage of churn the same across all services?
2. Is there some kind of client more likely to churn?

## Data:

For this analysis, we'll be exploring the Telco Customer Churn DataSet, where each row represents a customer, each column contains customer’s attributes, such as:


* Customer account information, e.g. how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges.

* Demographic info about customers, e.g. gender, age range, and if they have partners and dependents

* Services that each customer has signed up for e.g. phone, multiple lines, etc.


## Data Preparation

### Set-Up

We'll start by importing the necessary libraries and data.

<b><font color='red'>Important</font></b>: In order to have a cleaner version of the code, we have created a package to wrap all the helper functions used to analyze the data. You can access the helper functions by opening the  "./utilities/functions.py" script, where each function has been carefully documented. 

In [1]:
import pandas as pd
from utilities.functions import count_null_entries

In [2]:
df = pd.read_csv("./data/telcocustomerchurn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
print("Total number of rows: {}\nTotal number of columns: {}".format(df.shape[0],df.shape[1]))

Total number of rows: 7043
Total number of columns: 21


Based on the previous cell we have 21 attributes of about nearly 7k customers.

### Missing values


Now that we know how many customers and attributes we have, we are going to analyze if there is some missing or zero variance data.

The below cell shows the result of `count_null_entries()` function, this function returns the number of null entries per column as a pandas DataFrame,  feel free to examine the code within the `utilities package`.

In [4]:
count_null_entries(df).head()

,Col_Name,Num_Nulls
0,customerID,0
1,gender,0
2,SeniorCitizen,0
3,Partner,0
4,Dependents,0


In [5]:
null_entries = count_null_entries(df)
null_entries[null_entries['Num_Nulls']>0]

,Col_Name,Num_Nulls


From the above cell, we can conclude that there is no missing data at any column.

Definitely, this is not a common situation, but for now, we can continue.

### Non-Variance Columns 

Before starting to analyze whether a column has the right amount of variability to be useful, we would like to separate the following steps for categorical and numerical variables.

#### Categorical

In order to analyze the variability within the categorical variables, we:

1. Created `df_cat`, which is a DataFrame containing all the categorical variables.
2. Executed the `get_cat_summary()` function, which returns for each categorical variable:
    + BiggestCat: The most frequent category,
    + NumLevels: Total number of categories within the variable, e.g. (Yes/No = 2),
    + BiggestShare: The frequency of the most frequent category.
    
For instance, the **gender** variable has **2** possible categories, the most frequent category is **Male** which represents **50.48%** of the customers.

In [6]:
df_cat = df.select_dtypes(include='object').copy()

In [7]:
from utilities.functions import get_cat_summary
cat_summary = get_cat_summary(df_cat)
cat_summary.head()

,ColName,BiggestCat,NumLevels,BiggestShare
0,customerID,0002-ORFBO,7043,0.0001
1,gender,Male,2,0.5048
2,Partner,No,2,0.5170
3,Dependents,No,2,0.7004
4,PhoneService,Yes,2,0.9032


In [8]:
cat_summary.sort_values('BiggestShare', ascending = False).head(5)

,ColName,BiggestCat,NumLevels,BiggestShare
4,PhoneService,Yes,2,0.9032
17,Churn,No,2,0.7346
3,Dependents,No,2,0.7004
14,PaperlessBilling,Yes,2,0.5922
13,Contract,Month-to-month,3,0.5502


From the above table, we can see that there are no non-variance columns, although the `PhoneService` column has the fewer variance (~90% of the entries belongs to the same category) it still might be interesting analyze it.

In [9]:
cat_summary.sort_values('BiggestShare').head(5)

,ColName,BiggestCat,NumLevels,BiggestShare
0,customerID,0002-ORFBO,7043,0.0001
16,TotalCharges,,6531,0.0016
15,PaymentMethod,Electronic check,4,0.3358
12,StreamingMovies,No,3,0.3954
11,StreamingTV,No,3,0.3990


On the other hand, seems that:

* customerID: this variable has as many categories as total rows, and for its name is pretty clear that this will not provide any useful information to answer our questions.

* TotalCharges: seems that this column does not suppose to be a categorical variable, so We'll analyze it in more detail.

finally, the rest of the columns seemed to be nice candidates to answer our questions.

#### Numerical

Before to start, let's cast `TotalCharges` to a numerical value

In [10]:
try:
    df['TotalCharges'].astype('float')
except ValueError:
    print("Could not convert some value to float")
    print("Missing values in {} row(s)".format(df[df['TotalCharges'].str.strip()==""].shape[0]))

Could not convert some value to float
Missing values in 11 row(s)


From the above cells, we can conclude that we have some missing `TotalCharges` for 11 customers. Let's take a deeper look at what might be happening. 

After  looking at the metadata, we have found the following:

* tenure: Number of months the customer has stayed with the company
* MonthlyCharges: The amount charged to the customer monthly
* TotalCharges: The total amount charged to the customer 

In most cases $TotalCharges \approx tenure*MonthlyCharges$

Hence, it's reasonable to assume that the missing data in `TotalCharges` are actually zeros.

In [11]:
print("----Examples of rows with missing ´TotalCharges´----")
display(df[df['TotalCharges'].str.strip()==""].head()[['tenure','MonthlyCharges','TotalCharges']])
print("----Examples of rows with data in ´TotalCharges´----")
display(df.head()[['tenure','MonthlyCharges','TotalCharges']])

----Examples of rows with missing ´TotalCharges´----


,tenure,MonthlyCharges,TotalCharges
488,0,52.55,
753,0,20.25,
936,0,80.85,
1082,0,25.75,
1340,0,56.05,


----Examples of rows with data in ´TotalCharges´----


,tenure,MonthlyCharges,TotalCharges
0,1,29.85,29.85
1,34,56.95,1889.5
2,2,53.85,108.15
3,45,42.30,1840.75
4,2,70.70,151.65


Let's apply what we have concluded and convert the `TotalCharges` column to numeric.

In [12]:
df['TotalCharges'] = df['TotalCharges'].apply(lambda x: "0" if x.strip()=="" else x)
try:
    df['TotalCharges'] = df['TotalCharges'].astype('float')
except ValueError:
    print("Could not convert some value to float")

Now we can continue:

In [13]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_num = df.select_dtypes(include=numerics)

In [14]:
df_num.describe().transpose().apply(lambda x: round(x,2))

,count,mean,std,min,25%,50%,75%,max
SeniorCitizen,7043.0,0.16,0.37,0.00,0.00,0.00,0.00,1.00
tenure,7043.0,32.37,24.56,0.00,9.00,29.00,55.00,72.00
MonthlyCharges,7043.0,64.76,30.09,18.25,35.50,70.35,89.85,118.75
TotalCharges,7043.0,2279.73,2266.79,0.00,398.55,1394.55,3786.60,8684.80


Seems that everything it's ok except for `SeniorCitizen` column, which is more likely to be a categorical variable.

In [15]:
df['SeniorCitizen'] = df['SeniorCitizen'].apply(lambda x: "No" if x == 0 else "Yes")

## Exploratory Data Analysis

Now that we have cleaned the data and now that we have a better understanding of the variables at hand, let's start by answering the first question.

But first, we need to perform one more step,  converting the `Churn` column into something that can be easily analyzed.

In [16]:
from utilities.functions import compare_churn_prob
df['Churn'] = df['Churn'].apply(lambda x: 1 if x =="Yes" else 0)

Going back to our first question:

**<center>Is the percentage of churn the same across all services?</center>**

In order to have a clear definition of what does it mean service in this context, we have selected the following attributes to be analyzed:

* PhoneService,
* InternetService,
* StreamingTV,
* StreamingMovies.

For each of the earlier mention attributes, we will be performing the following steps to decide whether the churn probability behaves differently:

1. Select a service, e.g. `PhoneService,
2. Group all customers across the selected service's  categories
3. Calculate for each category:
    + `Count`: Total number of customers
    + `Churn Abs.`: Total number of observed churn
    + `Churn Prob`: Probability of churn $Churn Abs./Count$ 
    + `Upper and Lower Boundary`: Confidence interval's upper and lower boundaries, under the z-norm assumption
4. Decide if the observed difference between the probabilities of two levels is significant, e.g. by not having overlapping confidence intervals or by comparing with the mean value 26.53%  

Let's start!

**PhoneService**:

We can see that there is no significant difference between churn probability by having or not a phone service. 

In [19]:
df_z_test = compare_churn_prob(df, 'PhoneService')
display(df_z_test)

,PhoneService,p_churn,churn,count,ci_low,ci_upp
0,No,0.249267,170,682,0.216801,0.281733
1,Yes,0.267096,1699,6361,0.256224,0.277969


**InternetService**:

Here we have some interesting results:

* Fiber optic: Has the highest churn probability, 41.8%. Probably there something about the service that we can improve in order to retain more customers. We might be not accomplishing our customer's expectation in term of internet speed.

* DSL: The probability, as well as their confidence interval, are under the mean churn probability, however, from a business perspective we might be wanting to turn this type of service into Fiber Optic.

* No Internet Service: Th category with the lowest churn probability, this might be interpreted as a clear sign that we have to improve our Internet Service.

In [21]:
df_z_test = compare_churn_prob(df, 'InternetService')
display(df_z_test)

,InternetService,p_churn,churn,count,ci_low,ci_upp
0,DSL,0.189591,459,2421,0.173977,0.205205
1,Fiber optic,0.418928,1297,3096,0.401548,0.436307
2,No,0.074050,113,1526,0.060912,0.087188
